<h2> Chatbot for Dental Office</h2>

In [31]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
from bs4 import BeautifulSoup

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:2]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:2]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk
Anthropic API Key exists and begins sk
Google API Key exists and begins AI


In [14]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [42]:
#scrape website

def get_website_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Remove script and style elements
        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        # Extract visible text
        text = soup.get_text(separator="\n")
        
        # Clean up whitespace
        lines = [line.strip() for line in text.splitlines()]
        content = "\n".join(line for line in lines if line)

        return content[:100000]  
    except Exception as e:
        return f"Error fetching website content: {e}"

In [43]:
url = "https://rooseveltislandadvanceddentistry.com/"
website_content = get_website_text(url)

In [44]:
website_content

"Roosevelt Island Advanced Dentistry | Pediatric Dentist\nBusiness Hours:\nMonday to Saturday 9 AM to 5 PM\n(212) 752-8722\n501A Main St, New York, NY 10044, United States\nRoosevelt Island\ncontact us\nBook an appointment\nOur Team\nPediatric Procedures\nPediatric Services\nPediatric Topics\nProsthodontics Procedures\nDental Implants\nGeneral Dentistry\nServices\nCosmetic Dentistry\nOral Hygiene\nEmergency\nOrthodontics\nOnline Forms\nTestimonials\nOur Team\nPediatric Procedures\nPediatric Services\nPediatric Topics\nProsthodontics Procedures\nDental Implants\nGeneral Dentistry\nServices\nCosmetic Dentistry\nOral Hygiene\nEmergency\nOrthodontics\nOnline Forms\nTestimonials\nBusiness Hours:\nMonday to Saturday 9 AM to 5 PM\n(212) 752-8722\n501A Main St, New York, NY 10044, United States\nRoosevelt Island\ncontact us\nBook an appointment\nNEW YORK PEDIATRIC DENTIST\nThe best and only Pediatric Dentist Specialist in Roosevelt Island, NY.\nThe best and only Pediatric Dentist Specialist in

In [45]:
system_message = (
    "You are a helpful assistant for a dental office. Your job is to welcome the patient. Use the information below to answer patient questions "
    "about services, office hours, insurance, contact details, doctors and booking appointments.\n\n"
    f"Website content:\n{website_content}"
)

In [46]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [47]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
